In [19]:
"""This is NoteBook File that i created for Get Prediction from each image"""

'This is NoteBook File that i created for Get Prediction from each image'

In [1]:
#Installing Dependacies
# !pip install -U layoutparser
# !pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
# !pip install layoutparser[ocr]


In [2]:
import layoutparser as lp
import cv2
import os
from matplotlib import pyplot as plt
import torch

In [3]:
## Extract Title from the Image and return image with those Coordinates

def doc_title_segement(file_path):
    
    
    # Configurations
    # HJDataset	{1:"Page Frame", 2:"Row", 3:"Title Region", 4:"Text Region", 5:"Title", 6:"Subtitle", 7:"Other"}
    # PubLayNet	{0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
    # PrimaLayout	{1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}
    # NewspaperNavigator	{0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}
    # TableBank	{0: "Table"}
    
    # Loading Layout Parser Model
    model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.65],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
    
    # This will store crop title from images 
    # Give Benefit at batch processing in yolo
    crop_image_title = []
    
    for file_path in os.scandir(file_path):
        if file_path.is_file():
            
            #Read image and convert into RGB
            image = cv2.imread(file_path.path)[..., ::-1]
            
            layout = model.detect(image)
            
            
            # Extract Title Region Only
            text_blocks = lp.Layout([b for b in layout if b.type=="Title"]) 
            
            
            figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])
            
            
            text_blocks = lp.Layout([b for b in text_blocks \
                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
            
            
            
            h, w = image.shape[:2]

            left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

            left_blocks = text_blocks.filter_by(left_interval, center=True)
            left_blocks.sort(key = lambda b:b.coordinates[1])

            right_blocks = [b for b in text_blocks if b not in left_blocks]
            right_blocks.sort(key = lambda b:b.coordinates[1])

            # And finally combine the two list and add the index
            # according to the order
            text_blocks = lp.Layout([b.set(id = idx) 
                                     for idx, b in 
                                     enumerate(left_blocks + right_blocks)])
            
            
            #
                    
            
            
            for dic in text_blocks.to_dict()["blocks"]:
                
                # Extract x1,y1,x_2,y_2 From dictionary
                x1,y1,x2,y2 = dic["x_1"],dic["y_1"],dic["x_2"],dic["y_2"]
                
                print(x1,y1,x2,y2)
                
                
                
                crop_image_title.append(image[int(y1-3):int(y2+3),int(x1-3):int(x2+3)])
                
                
        return crop_image_title
                
            
            
            
            

In [19]:
# Get Array Of images and predict each of image using YOlo v5
def yolo_model_heading_detection(crop_title_array):
    
    
    # Best Weight Model Path
    model_path = "/home/nipun/Documents/NIpun/Git_Repos/Document-Segmentation/Doc_Segemnt/best.pt"
    
    
    # In order to load model from local need to add  source='local'
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path,force_reload=True)
    
    
    # Add Test time augmentation for better prediction
    results = model(crop_images, size=640,augment=True)
    
    
    
    # If we need we can return panda dataframe
    # results.pandas().xyxy[0]
        
    
    # To Json
    # results.pandas().xyxy[0].to_json(orient="records")
    
    # To save Results
    results.save()
    
    
    return results.print()
    

In [5]:

#This will extract content from image and will return dictionary
def doc_text_segement(file_path):
    
    
    # Configurations
    # HJDataset	{1:"Page Frame", 2:"Row", 3:"Title Region", 4:"Text Region", 5:"Title", 6:"Subtitle", 7:"Other"}
    # PubLayNet	{0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
    # PrimaLayout	{1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}
    # NewspaperNavigator	{0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}
    # TableBank	{0: "Table"}
    
    # Loading Layout Parser Model
    model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.65],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
    
    
    text_array = []
    
    for file_path in os.scandir(file_path):
        if file_path.is_file():
            
            #Read image and convert into RGB
            image = cv2.imread(file_path.path)[..., ::-1]
            
            layout = model.detect(image)
            
            
            
            # Extract Text Region Only
            text_blocks = lp.Layout([b for b in layout if b.type=="Text"]) 
            
            
            figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])
            
            
            text_blocks = lp.Layout([b for b in text_blocks \
                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
            
            
            
            h, w = image.shape[:2]

            left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

            left_blocks = text_blocks.filter_by(left_interval, center=True)
            left_blocks.sort(key = lambda b:b.coordinates[1])

            right_blocks = [b for b in text_blocks if b not in left_blocks]
            right_blocks.sort(key = lambda b:b.coordinates[1])

            # And finally combine the two list and add the index
            # according to the order
            text_blocks = lp.Layout([b.set(id = idx) 
                                     for idx, b in 
                                     enumerate(left_blocks + right_blocks)])
            
            
            
            # Append Coordinates into Text Array
            text_array.append(text_blocks.to_dict()["blocks"])
                    
            
            

                
                
        return text_array
                
            
            
            
            

In [16]:
doc_text_segement("./Images")


Loading checkpoint from https://www.dropbox.com/s/57zjbwv6gh3srry/model_final.pth?dl=1


[[{'x_1': 83.01058959960938,
   'y_1': 534.6427001953125,
   'x_2': 171.05123901367188,
   'y_2': 551.2643432617188,
   'block_type': 'rectangle',
   'id': 0,
   'type': 'Text',
   'score': 0.8441960215568542},
  {'x_1': 54.133460998535156,
   'y_1': 18.816341400146484,
   'x_2': 908.9793090820312,
   'y_2': 92.06700134277344,
   'block_type': 'rectangle',
   'id': 1,
   'type': 'Text',
   'score': 0.8043978214263916},
  {'x_1': 41.26341247558594,
   'y_1': 164.6133575439453,
   'x_2': 886.8906860351562,
   'y_2': 235.06654357910156,
   'block_type': 'rectangle',
   'id': 2,
   'type': 'Text',
   'score': 0.7681299448013306},
  {'x_1': 32.65528106689453,
   'y_1': 113.68264770507812,
   'x_2': 880.6586303710938,
   'y_2': 154.24501037597656,
   'block_type': 'rectangle',
   'id': 3,
   'type': 'Text',
   'score': 0.746097981929779},
  {'x_1': 75.462158203125,
   'y_1': 247.4720001220703,
   'x_2': 845.6636962890625,
   'y_2': 265.3075866699219,
   'block_type': 'rectangle',
   'id': 4,

In [17]:
crop_images=doc_title_segement("./Images")

Loading checkpoint from https://www.dropbox.com/s/57zjbwv6gh3srry/model_final.pth?dl=1


82.91815185546875 437.9176940917969 503.8951416015625 455.52581787109375
640.8719482421875 577.3986206054688 871.2575073242188 675.0


In [20]:
yolo_model_heading_detection(crop_images)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/nipun/.cache/torch/hub/master.zip
YOLOv5 🚀 2021-8-9 torch 1.7.1 CPU

Fusing layers... 
Model Summary: 224 layers, 7059304 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 
Saved 2 images to 'runs/detect/exp'
image 1/2: 24x427 2 h3s
image 2/2: 101x237 (no detections)
Speed: 1.4ms pre-process, 141.0ms inference, 0.5ms NMS per image at shape (2, 3, 288, 640)


In [13]:
def doc_image_segement(file_path):
    
    
    #ConfigurationS
    # HJDataset	{1:"Page Frame", 2:"Row", 3:"Title Region", 4:"Text Region", 5:"Title", 6:"Subtitle", 7:"Other"}
    # PubLayNet	{0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
    # PrimaLayout	{1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}
    # NewspaperNavigator	{0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}
    # TableBank	{0: "Table"}
    
    # Loading Layout Parser Model
    model = lp.Detectron2LayoutModel('lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config',extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.65],
                                 label_map={1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"})
    
    # This will returrn location of the images
    image_location_array = []
    
    for file_path in os.scandir(file_path):
        
        
        # Read Images as RGB
        image = cv2.imread(file_path.path)[..., ::-1]
        
        layout = model.detect(image) 
        
        
        image_blocks = lp.Layout([b for b in layout if b.type=="ImageRegion"])
        
        
        image_location_array.append(image_blocks.to_dict()["blocks"])
        
        
    return image_location_array
        

In [14]:
doc_image_segement("./Images")


Loading checkpoint from https://www.dropbox.com/s/h7th27jfv19rxiy/model_final.pth?dl=1
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[[], []]